In [ ]:
import tensorflow as tf

In [ ]:
# processing input training data
feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(XTrain)

# makes a deep neural network with 2 hidden layers with 512 neurons each, and 10 output neurons
dnn_clf=tf.contrib.learn.DNNClassifier(hidden_units=[512,512], n_classes=10, feature_columns=feature_columns)

# fits the DNN to the training data using the training labels
dnn_clf.fit(x=XTrain, y=yi, batch_size=50, steps=40000)

# evaluates the performance of the DNN on the training data
dnn_clf.evaluate(XTrain, yi)